In [32]:
import os
os.environ["TRANSFORMERS_OFFLINE"] = "1"
os.environ["HF_DATASETS_OFFLINE"] = "1"


In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = "/workspace/AAIPL/hf_models/huggingface/models--microsoft--Phi-4-mini-instruct/snapshots/cfbefacb99257ffa30c83adab238a50856ac3083"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    local_files_only=True,
    trust_remote_code=False,  # 🔥 Important
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    local_files_only=True,
    trust_remote_code=False,  # 🔥 Important
)

print("✅ Loaded without dynamic module writing")


`torch_dtype` is deprecated! Use `dtype` instead!
[2026-02-14 11:36:34] INFO modeling.py:987: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Loaded without dynamic module writing


In [34]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 3,145,728 || all params: 3,839,167,488 || trainable%: 0.0819


In [36]:
inputs = tokenizer(
    "Generate a difficult question about reinforcement learning.",
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Generate a difficult question about reinforcement learning. Your response must have 3 sections. Mark the beginning of each section with SECTION X, such as: SECTION 1. The word "agent" should appear at least 5 times. SECTION 1

In the context of reinforcement learning, consider


In [37]:
pip install trl accelerate



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
model.gradient_checkpointing_enable()
model.config.use_cache = False


In [39]:
import re
import torch

def reward_function(samples):
    rewards = []
    
    for text in samples:
        reward = 0
        
        # Reward if ends with question mark
        if text.strip().endswith("?"):
            reward += 1.0
        
        # Penalize if contains explanation words
        if "because" in text.lower():
            reward -= 0.5
        
        # Penalize formatting
        if "SECTION" in text:
            reward -= 1.0
        
        # Reward length window
        token_count = len(text.split())
        if 15 <= token_count <= 40:
            reward += 1.0
        
        # Reward advanced terms
        advanced_terms = ["convergence", "approximation", "instability", 
                          "divergence", "stochastic", "policy gradient"]
        
        for term in advanced_terms:
            if term in text.lower():
                reward += 0.5
        
        rewards.append(reward)
    
    return torch.tensor(rewards)


In [49]:
import torch
import json
import time
import re
from typing import List
from unsloth import FastLanguageModel

torch.random.manual_seed(0)

MODEL_PATH = "/workspace/AAIPL/hf_models/Meta-Llama-3.1-8B-Instruct"
OUTPUT_FILE = "generated_reasoning_dataset.jsonl"


In [50]:
ALPACA_PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


In [51]:
class QAgent:
    def __init__(self, max_seq_length=4096):
        dtype = None
        load_in_4bit = False

        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name=MODEL_PATH,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit,
            device_map="cuda",
        )

        FastLanguageModel.for_inference(self.model)
        print("✅ QAgent model loaded")

    def generate(self, instruction: str, max_new_tokens=800):
        prompt = ALPACA_PROMPT.format(instruction, "")
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        outputs = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.9,
            top_p=0.95,
            do_sample=True,
            use_cache=True,
        )

        generated_ids = outputs[0][len(inputs.input_ids[0]):]
        return self.tokenizer.decode(generated_ids, skip_special_tokens=True).strip()


In [52]:
GENERATION_INSTRUCTION = """
Generate one difficult logical reasoning MCQ.

Allowed topics:
- Syllogism
- Seating Arrangement (Circular or Linear, no permutation counting)
- Blood Relations
- Mixed Alphanumeric Series

Return ONLY valid JSON in this exact format:

{
    "topic": "One of the allowed topics",
    "question": "Full question ending with a question mark?",
    "choices": [
        "A) ...",
        "B) ...",
        "C) ...",
        "D) ..."
    ],
    "answer": "A/B/C/D",
    "explanation": "Brief explanation"
}

Rules:
- Must require multi-step reasoning
- Only one correct answer
- No ambiguity
- No extra commentary outside JSON
"""


In [53]:
def extract_json(text):
    try:
        json_match = re.search(r"\{.*\}", text, re.DOTALL)
        if json_match:
            return json.loads(json_match.group())
    except:
        return None
    return None


In [54]:
def generate_dataset(n_samples=100):
    model = QAgent()
    dataset = []

    for i in range(n_samples):
        try:
            response = model.generate(GENERATION_INSTRUCTION)
            data = extract_json(response)

            if data:
                dataset.append(data)
                print(f"✅ {i+1}/{n_samples} generated")
            else:
                print(f"❌ Invalid JSON at {i+1}")

        except Exception as e:
            print(f"⚠ Error at {i+1}: {e}")

    return dataset


In [55]:
def save_dataset(dataset, filename):
    with open(filename, "w") as f:
        for item in dataset:
            f.write(json.dumps(item) + "\n")
    print(f"💾 Saved {len(dataset)} samples to {filename}")


In [57]:
import os

for root, dirs, files in os.walk("/workspace/AAIPL/hf_models"):
    if "config.json" in files:
        print(root)


/workspace/AAIPL/hf_models/huggingface/models--microsoft--Phi-4-mini-instruct/snapshots/cfbefacb99257ffa30c83adab238a50856ac3083
/workspace/AAIPL/hf_models/huggingface/models--Unsloth--Llama-3.1-8B-Instruct/snapshots/4699cc75b550f9c6f3173fb80f4703b62d946aa5
/workspace/AAIPL/hf_models/huggingface/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/c170c708c41dac9275d15a8fff4eca08d52bab71
/workspace/AAIPL/hf_models/huggingface/models--unsloth--gpt-oss-20b-BF16/snapshots/cc89b3e7fd423253264883a80a4fa5abc619649f
/workspace/AAIPL/hf_models/huggingface/models--google--gemma-3-12b-it/snapshots/96b6f1eccf38110c56df3a15bffe176da04bfd80
/workspace/AAIPL/hf_models/huggingface/models--Qwen--Qwen2.5-14B-Instruct/snapshots/cf98f3b3bbb457ad9e2bb7baf9a0125b6b88caa8
/workspace/AAIPL/hf_models/huggingface/models--Qwen--Qwen3-4B/snapshots/1cfa9a7208912126459214e8b04321603b3df60c


In [58]:
import torch
from unsloth import FastLanguageModel

MODEL_PATH = "/workspace/AAIPL/hf_models/huggingface/models--Unsloth--Llama-3.1-8B-Instruct/snapshots/4699cc75b550f9c6f3173fb80f4703b62d946aa5"

class QAgent:
    def __init__(self, max_seq_length=4096):
        dtype = torch.bfloat16  # AMD safe
        load_in_4bit = False

        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name=MODEL_PATH,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit,
            device_map="auto",
        )

        FastLanguageModel.for_inference(self.model)
        print("✅ QAgent loaded successfully")

    def generate(self, instruction, max_new_tokens=800):
        prompt = f"""Below is an instruction.

### Instruction:
{instruction}

### Response:
"""
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        outputs = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.9,
            top_p=0.95,
            do_sample=True,
            use_cache=True,
        )

        generated_ids = outputs[0][len(inputs.input_ids[0]):]
        return self.tokenizer.decode(generated_ids, skip_special_tokens=True).strip()


In [59]:
model = QAgent()

response = model.generate(
    "Generate one difficult syllogism MCQ in valid JSON format.",
    max_new_tokens=600
)

print(response)


[2026-02-14 12:03:46] INFO modeling.py:987: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
==((====))==  Unsloth 2025.10.9: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.11.1rc3.dev39+gf417746ad.rocm700.
   \\   /|    . Num GPUs = 1. Max memory: 255.688 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0a0+git1c57644. ROCm Toolkit: 7.0.51831-a3e329ad8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ QAgent loaded successfully
Here is a difficult syllogism MCQ in valid JSON format:

```json
{
  "question": "All humans are mortal. Socrates is human. Therefore, Socrates is mortal. Which of the following is a valid conclusion?",
  "options": [
    {
      "text": "All humans are mortal, so Socrates is mortal",
      "correct": false
    },
    {
      "text": "Socrates is a human, so he is definitely mortal",
      "correct": false
    },
    {
      "text": "Socrates is mortal because all humans are mortal",
      "correct": true
    },
    {
      "text": "Humans are mortal only if Socrates is too",
      "correct": false
    }
  ]
}
```

### Explanation:
This MCQ presents a syllogism based on the principle of universal instantiation, which is a valid logical argument. The correct answer is a straightforward application of this principle. The other options either rephrase the premises without logically connecting them, or make unfounded assumptions, which are characteristic of inv

In [60]:
import re
import json

def extract_json(text):
    try:
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            return json.loads(match.group())
    except:
        return None
    return None


In [61]:
def validate_sample(sample):
    required_keys = ["topic", "question", "choices", "answer", "explanation"]

    # Check structure
    if not all(k in sample for k in required_keys):
        return False

    # Must have exactly 4 choices
    if len(sample["choices"]) != 4:
        return False

    # Question must end with ?
    if not sample["question"].strip().endswith("?"):
        return False

    # No permutation counting
    if "permutation" in sample["question"].lower():
        return False

    # Only one answer letter
    if sample["answer"] not in ["A", "B", "C", "D"]:
        return False

    return True


In [62]:
GENERATION_INSTRUCTION = """
Generate one difficult logical reasoning MCQ.

Allowed topics:
- Syllogism
- Seating Arrangement (Circular or Linear, no permutation counting)
- Blood Relations
- Mixed Alphanumeric Series

Return ONLY valid JSON in this format:

{
    "topic": "...",
    "question": "...?",
    "choices": ["A) ...", "B) ...", "C) ...", "D) ..."],
    "answer": "A/B/C/D",
    "explanation": "Brief explanation"
}

Rules:
- Multi-step reasoning required
- Only one correct answer
- No ambiguity
- No extra commentary outside JSON
"""


In [63]:
def generate_dataset(model, target_size=1000):
    dataset = []
    attempts = 0
    max_attempts = target_size * 3  # allow retries

    while len(dataset) < target_size and attempts < max_attempts:
        attempts += 1
        
        response = model.generate(GENERATION_INSTRUCTION, max_new_tokens=700)
        sample = extract_json(response)

        if sample and validate_sample(sample):
            dataset.append(sample)
            print(f"✅ {len(dataset)}/{target_size} valid")
        else:
            print(f"❌ Invalid sample (attempt {attempts})")

    return dataset


In [64]:
def save_dataset(dataset, filename):
    with open(filename, "w") as f:
        for item in dataset:
            f.write(json.dumps(item) + "\n")
    print(f"💾 Saved {len(dataset)} samples to {filename}")


In [65]:
model = QAgent()

dataset = generate_dataset(model, target_size=20)
save_dataset(dataset, "test_reasoning.jsonl")


[2026-02-14 12:31:22] INFO modeling.py:987: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
==((====))==  Unsloth 2025.10.9: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.11.1rc3.dev39+gf417746ad.rocm700.
   \\   /|    . Num GPUs = 1. Max memory: 255.688 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0a0+git1c57644. ROCm Toolkit: 7.0.51831-a3e329ad8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ QAgent loaded successfully
❌ Invalid sample (attempt 1)
✅ 1/20 valid
❌ Invalid sample (attempt 3)
❌ Invalid sample (attempt 4)
✅ 2/20 valid
❌ Invalid sample (attempt 6)
❌ Invalid sample (attempt 7)
❌ Invalid sample (attempt 8)
❌ Invalid sample (attempt 9)
❌ Invalid sample (attempt 10)
❌ Invalid sample (attempt 11)
❌ Invalid sample (attempt 12)
✅ 3/20 valid
✅ 4/20 valid
✅ 5/20 valid
❌ Invalid sample (attempt 16)
✅ 6/20 valid
❌ Invalid sample (attempt 18)
✅ 7/20 valid
✅ 8/20 valid
✅ 9/20 valid
❌ Invalid sample (attempt 22)
❌ Invalid sample (attempt 23)
❌ Invalid sample (attempt 24)
✅ 10/20 valid
❌ Invalid sample (attempt 26)
✅ 11/20 valid
❌ Invalid sample (attempt 28)
❌ Invalid sample (attempt 29)
❌ Invalid sample (attempt 30)
❌ Invalid sample (attempt 31)
✅ 12/20 valid
❌ Invalid sample (attempt 33)
❌ Invalid sample (attempt 34)
✅ 13/20 valid
❌ Invalid sample (attempt 36)
❌ Invalid sample (attempt 37)
❌ Invalid sample (attempt 38)
✅ 14/20 valid
❌ Invalid sample (attempt 40)
✅ 15/20 val

In [66]:
print("Hello")

Hello


In [70]:
from datasets import load_dataset

DATA_FILE = "AAIPL/agents/test_reasoning.jsonl"

dataset = load_dataset("json", data_files=DATA_FILE, split="train")

def format_example(example):
    choices_text = "\n".join(example["choices"])
    
    return {
        "text": f"""### Instruction:
Topic: {example["topic"]}
Generate one difficult logical reasoning MCQ.
Do not provide explanation.
Return only the question and 4 choices.

### Response:
{example["question"]}

{choices_text}

Answer:
{example["answer"]}"""
    }

dataset = dataset.map(format_example)

print("Converted dataset size:", len(dataset))
print(dataset[0]["text"][:10])


FileNotFoundError: Unable to find '/workspace/AAIPL/agents/AAIPL/agents/test_reasoning.jsonl'

In [71]:
pip install pdfplumber requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 65.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 228.0 MB/s  0:00:00
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20250506
    Uninstalling pdfminer.six-20250506:
      Successfully uninstalled pdfminer.six-20250506
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pdfplumber]3 [pdfminer.six]

[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [76]:
import requests
import pdfplumber
import json
import re
import os

PDF_URLS = [
    "https://www.smartkeeda.com/pdf/N/Blood_Relation_PDF_Set_1.pdf",
    "https://www.smartkeeda.com/pdf/N/Blood_Relation_PDF_Set_2.pdf"
]

OUTPUT_FILE = "blood_relation_final_dataset.jsonl"

# -------------------------------------------------
# STEP 1: Download PDF
# -------------------------------------------------
def download_pdf(url, filename):
    print(f"⬇️ Downloading {url}")
    r = requests.get(url)
    r.raise_for_status()
    with open(filename, "wb") as f:
        f.write(r.content)
    return filename

# -------------------------------------------------
# STEP 2: Extract Text
# -------------------------------------------------
def extract_text(pdf_path):
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                full_text += text + "\n"
    return full_text

# -------------------------------------------------
# STEP 3: Extract Answer Grid
# -------------------------------------------------
def extract_answers(text):
    answer_block = re.search(r"CORRECT ANSWERS:(.*)", text, re.DOTALL | re.IGNORECASE)
    if not answer_block:
        print("❌ Answer block not found")
        return {}

    answer_text = answer_block.group(1)
    answers = re.findall(r"\b[A-E]\b", answer_text)

    answer_map = {}
    for i, ans in enumerate(answers, start=1):
        answer_map[i] = ans

    print(f"✅ Extracted {len(answer_map)} answers")
    return answer_map

# -------------------------------------------------
# STEP 4: Extract Explanation Blocks
# Handles: Common Explanation : [Q1 to Q3]
# -------------------------------------------------
def extract_explanations(text):
    explanation_map = {}

    pattern = re.compile(
        r"Common Explanation\s*:\s*\[Q?(\d+)\s*to\s*Q?(\d+)\](.*?)(?=\n\d+[\.\)]|\Z)",
        re.DOTALL | re.IGNORECASE
    )

    matches = pattern.findall(text)

    for start, end, explanation in matches:
        for q_num in range(int(start), int(end) + 1):
            explanation_map[q_num] = explanation.strip()

    print(f"✅ Extracted explanations for {len(explanation_map)} questions")
    return explanation_map

# -------------------------------------------------
# STEP 5: Split Merged Options
# Example: A. Mother B. Father C. Uncle ...
# -------------------------------------------------
def split_options(line):
    options = re.findall(r"[A-E][\.\)]\s*[^A-E]+(?=\s+[A-E][\.\)]|\Z)", line)
    return [opt.strip() for opt in options]

# -------------------------------------------------
# STEP 6: Extract Questions + Options
# -------------------------------------------------
def extract_questions(text):
    lines = text.split("\n")
    data = []

    current_question = None
    current_choices = []
    current_number = None

    for line in lines:
        line = line.strip()

        # Detect question start (1. or 1))
        q_match = re.match(r"^(\d+)[\.\)]\s*(.*)", line)
        if q_match:
            if current_question and current_choices:
                data.append({
                    "number": current_number,
                    "question": current_question.strip(),
                    "choices": current_choices
                })

            current_number = int(q_match.group(1))
            current_question = q_match.group(2)
            current_choices = []
            continue

        # Detect options (even if merged in one line)
        if re.search(r"[A-E][\.\)]", line) and current_question:
            current_choices = split_options(line)
            continue

        # Append multiline question content
        if current_question and not current_choices:
            current_question += " " + line

    # Add last question
    if current_question and current_choices:
        data.append({
            "number": current_number,
            "question": current_question.strip(),
            "choices": current_choices
        })

    print(f"✅ Extracted {len(data)} questions")
    return data

# -------------------------------------------------
# STEP 7: Merge Everything
# -------------------------------------------------
def merge_data(questions, answer_map, explanation_map):
    final_data = []

    for q in questions:
        q_num = q["number"]

        if q_num in answer_map:
            final_data.append({
                "topic": "Blood Relations",
                "question": q["question"],
                "choices": q["choices"],
                "answer": answer_map[q_num],
                "explanation": explanation_map.get(q_num, "")
            })

    print(f"✅ Final mapped dataset size: {len(final_data)}")
    return final_data

# -------------------------------------------------
# STEP 8: Save JSONL
# -------------------------------------------------
def save_jsonl(data, filename):
    with open(filename, "w") as f:
        for item in data:
            f.write(json.dumps(item) + "\n")
    print(f"💾 Saved to {filename}")

# -------------------------------------------------
# MAIN PIPELINE
# -------------------------------------------------
def main():
    combined_dataset = []

    for idx, url in enumerate(PDF_URLS):
        pdf_filename = f"temp_{idx}.pdf"

        download_pdf(url, pdf_filename)
        text = extract_text(pdf_filename)

        answer_map = extract_answers(text)
        explanation_map = extract_explanations(text)
        questions = extract_questions(text)

        final_data = merge_data(questions, answer_map, explanation_map)

        combined_dataset.extend(final_data)

        os.remove(pdf_filename)

    print(f"\n🎯 TOTAL QUESTIONS COLLECTED: {len(combined_dataset)}")
    save_jsonl(combined_dataset, OUTPUT_FILE)

if __name__ == "__main__":
    main()


⬇️ Downloading https://www.smartkeeda.com/pdf/N/Blood_Relation_PDF_Set_1.pdf
✅ Extracted 143 answers
✅ Extracted explanations for 27 questions
✅ Extracted 46 questions
✅ Final mapped dataset size: 46
⬇️ Downloading https://www.smartkeeda.com/pdf/N/Blood_Relation_PDF_Set_2.pdf
✅ Extracted 116 answers
✅ Extracted explanations for 0 questions
✅ Extracted 49 questions
✅ Final mapped dataset size: 49

🎯 TOTAL QUESTIONS COLLECTED: 95
💾 Saved to blood_relation_final_dataset.jsonl


In [78]:
pip install pdf2image pytesseract pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pdf2image]

[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [80]:
!apt update
!apt install -y poppler-utils


Get:1 https://repo.radeon.com/amdgpu/7.0/ubuntu jammy InRelease [3183 B]
Get:2 https://repo.radeon.com/rocm/apt/7.0 jammy InRelease [2603 B]            
Get:3 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:5 https://repo.radeon.com/amdgpu/7.0/ubuntu jammy/main amd64 Packages [1329 B]m
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:8 https://repo.radeon.com/rocm/apt/7.0 jammy/main amd64 Packages [82.7 kB] 
Get:9 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]     
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]m
Get:11 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:12 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:13 http://archive.ubuntu.com/

In [81]:
!apt install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fontconfig libdatrie1 libfribidi0 libgif7 libgraphite2-3 libharfbuzz0b
  liblept5 libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libtesseract4
  libthai-data libthai0 libwebpmux3 tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  fontconfig libdatrie1 libfribidi0 libgif7 libgraphite2-3 libharfbuzz0b
  liblept5 libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libtesseract4
  libthai-data libthai0 libwebpmux3 tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 17 newly installed, 0 to remove and 64 not upgraded.
Need to get 8437 kB of archives.
After this operation, 25.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libfribidi0 amd64 1.0.8-2ubuntu3.1 [26.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fontconfig

In [82]:
!pip install pdf2image pytesseract pillow



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [83]:
from pdf2image import convert_from_path
import pytesseract
import os

PDF_PATH = "book_sample.pdf"  # make sure file exists
OUTPUT_FILE = "ocr_output.txt"

def extract_text_ocr(pdf_path):
    print("🔄 Converting PDF to images...")
    images = convert_from_path(pdf_path)

    full_text = ""
    for i, image in enumerate(images):
        print(f"OCR page {i+1}")
        text = pytesseract.image_to_string(image)
        full_text += text + "\n"

    return full_text

text = extract_text_ocr(PDF_PATH)

with open(OUTPUT_FILE, "w") as f:
    f.write(text)

print("✅ OCR extraction complete")
print("\nFirst 500 characters:\n")
print(text[:500])


🔄 Converting PDF to images...
OCR page 1
OCR page 2
OCR page 3
OCR page 4
OCR page 5
OCR page 6
OCR page 7
OCR page 8
OCR page 9
OCR page 10
OCR page 11
OCR page 12
OCR page 13
✅ OCR extraction complete

First 500 characters:

BLOOD RELATIONS

19

ea a

 

 

DEFINITION

Those persons who are related to another persons by birth rather than by marriage
are called in Blood Relation.

Let’s start right from basic and see what are the different blood relations in a family,
which we should know to understand the logical reasoning in blood relation
questions.

Remember the relations as given below:

(i) Children of Same parents | —>+ Siblings
One’s Husband or Wife —> Spouse
Relatives on Mother’s Side —» Maternal
Relatives o


In [88]:
# Q-Agent (Production Version)
import time
import torch
from typing import Optional, List
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

torch.random.manual_seed(0)

# 🔹 CHANGE THESE PATHS IF NEEDED
BASE_MODEL_PATH = "/workspace/AAIPL/hf_models/huggingface/models--Qwen--Qwen2.5-14B-Instruct/snapshots/cf98f3b3bbb457ad9e2bb7baf9a0125b6b88caa8"
Q_LORA_PATH = "/workspace/AAIPL/qwen14b_lora"   # Your fine-tuned Q-agent weights

class QAgent(object):
    def __init__(self, **kwargs):

        # ---------------- TOKENIZER ----------------
        self.tokenizer = AutoTokenizer.from_pretrained(
            BASE_MODEL_PATH,
            padding_side="left",
            local_files_only=True,
            trust_remote_code=True
        )

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        # ---------------- BASE MODEL ----------------
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_PATH,
            device_map="auto",
            torch_dtype=torch.float16,
            local_files_only=True,
            trust_remote_code=True
        )

        # ---------------- ATTACH FINETUNED LoRA ----------------
        self.model = PeftModel.from_pretrained(
            base_model,
            Q_LORA_PATH,
            is_trainable=False
        )

        self.model.eval()

In [ ]:
# Q-Agent (Production Version)
import time
import torch
from typing import Optional, List
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

torch.random.manual_seed(0)

# 🔹 CHANGE THESE PATHS IF NEEDED
BASE_MODEL_PATH = "/workspace/AAIPL/hf_models/huggingface/models--Qwen--Qwen2.5-14B-Instruct/snapshots/cf98f3b3bbb457ad9e2bb7baf9a0125b6b88caa8"
Q_LORA_PATH = "/workspace/AAIPL/qwen14b_lora"   # Your fine-tuned Q-agent weights

class QAgent(object):
    def __init__(self, **kwargs):

        # ---------------- TOKENIZER ----------------
        self.tokenizer = AutoTokenizer.from_pretrained(
            BASE_MODEL_PATH,
            padding_side="left",
            local_files_only=True,
            trust_remote_code=True
        )

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        # ---------------- BASE MODEL ----------------
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_PATH,
            device_map="auto",
            torch_dtype=torch.float16,
            local_files_only=True,
            trust_remote_code=True
        )

        # ---------------- ATTACH FINETUNED LoRA ----------------
        self.model = PeftModel.from_pretrained(
            base_model,
            Q_LORA_PATH,
            is_trainable=False
        )

        self.model.eval()

    def generate_response(
        self,
        message: str | List[str],
        system_prompt: Optional[str] = None,
        **kwargs
    ):

        if system_prompt is None:
            system_prompt = "You are a difficult reasoning question generator."

        if isinstance(message, str):
            message = [message]

        # Prepare chat format
        all_messages = []
        for msg in message:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": msg},
            ]
            all_messages.append(messages)

        # Apply chat template
        texts = []
        for messages in all_messages:
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
            )
            texts.append(text)

        # Tokenize
        model_inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(self.model.device)

        tgps_show = kwargs.get("tgps_show", False)

        if tgps_show:
            start_time = time.time()

        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=kwargs.get("max_new_tokens", 512),
            temperature=kwargs.get("temperature", 0.7),
            top_p=kwargs.get("top_p", 0.9),
            do_sample=kwargs.get("do_sample", True),
            pad_token_id=self.tokenizer.pad_token_id,
        )

        if tgps_show:
            generation_time = time.time() - start_time

        batch_outs = []
        token_len = 0

        for input_ids, generated_sequence in zip(model_inputs.input_ids, generated_ids):
            output_ids = generated_sequence[len(input_ids):]
            token_len += len(output_ids)

            content = self.tokenizer.decode(
                output_ids,
                skip_special_tokens=True
            ).strip()

            batch_outs.append(content)

        if tgps_show:
            return (
                batch_outs[0] if len(batch_outs) == 1 else batch_outs,
                token_len,
                generation_time
            )

        return batch_outs[0] if len(batch_outs) == 1 else batch_outs, None, None


# ---------------- TEST BLOCK ----------------
if __name__ == "__main__":

    q_agent = QAgent()

    response, tl, gt = q_agent.generate_response(
        """
        Generate a difficult Blood Relation MCQ question.
        Return valid JSON with topic, question, choices, answer and explanation.
        """,
        max_new_tokens=512,
        temperature=0.8,
        tgps_show=True
    )

    print("Response:\n", response)
    print(f"Tokens: {tl}, Time: {gt:.2f}s, TPS: {tl/gt:.2f}")


In [89]:
import time
import torch
from typing import Optional, List
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

torch.manual_seed(0)

BASE_MODEL_PATH = "/workspace/AAIPL/hf_models/huggingface/models--Qwen--Qwen2.5-14B-Instruct/snapshots/cf98f3b3bbb457ad9e2bb7baf9a0125b6b88caa8"
Q_LORA_PATH = "/workspace/AAIPL/qwen14b_lora"


class QAgent:
    def __init__(self):

        # ---------------- TOKENIZER ----------------
        self.tokenizer = AutoTokenizer.from_pretrained(
            BASE_MODEL_PATH,
            padding_side="left",
            local_files_only=True,
            trust_remote_code=True,
        )

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        # ---------------- BASE MODEL ----------------
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_PATH,
            device_map="auto",
            torch_dtype=torch.float16,
            local_files_only=True,
            trust_remote_code=True,
        )

        # ---------------- ATTACH LoRA ----------------
        self.model = PeftModel.from_pretrained(
            base_model,
            Q_LORA_PATH,
            is_trainable=False,
        )

        self.model.eval()
        torch.set_grad_enabled(False)

    # ✅ FIXED INDENTATION HERE
    def generate_response(
        self, message: str | List[str], system_prompt: Optional[str] = None, **kwargs
    ):

        if system_prompt is None:
            system_prompt = "You are a helpful assistant."

        if isinstance(message, str):
            message = [message]

        all_messages = []
        for msg in message:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": msg},
            ]
            all_messages.append(messages)

        texts = []
        for messages in all_messages:
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                enable_thinking=False,
            )
            texts.append(text)

        model_inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
        ).to(self.model.device)

        tgps_show_var = kwargs.get("tgps_show", False)

        if tgps_show_var:
            start_time = time.time()

        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=kwargs.get("max_new_tokens", 1024),
            pad_token_id=self.tokenizer.pad_token_id,
            temperature=kwargs.get("temperature", 0.7),
            top_p=kwargs.get("top_p", 0.9),
            do_sample=kwargs.get("do_sample", True),
        )

        if tgps_show_var:
            generation_time = time.time() - start_time

        batch_outs = []
        token_len = 0

        for input_ids, generated_sequence in zip(
            model_inputs.input_ids, generated_ids
        ):
            output_ids = generated_sequence[len(input_ids):].tolist()

            if tgps_show_var:
                token_len += len(output_ids)

            # remove thinking content if present
            if 151668 in output_ids:
                index = len(output_ids) - output_ids[::-1].index(151668)
            else:
                index = 0

            content = self.tokenizer.decode(
                output_ids[index:], skip_special_tokens=True
            ).strip("\n")

            batch_outs.append(content)

        if tgps_show_var:
            return (
                batch_outs[0] if len(batch_outs) == 1 else batch_outs,
                token_len,
                generation_time,
            )

        return batch_outs[0] if len(batch_outs) == 1 else batch_outs, None, None


# ---------------- TEST ----------------
if __name__ == "__main__":
    model = QAgent()

    response, tl, tm = model.generate_response(
        "Generate a hard MCQ on Number Series in JSON format.",
        tgps_show=True,
        max_new_tokens=512,
        temperature=0.1,
        top_p=0.9,
        do_sample=True,
    )

    print("Response:", response)
    print(
        f"Total tokens: {tl}, Time taken: {tm:.2f} seconds, TGPS: {tl/tm:.2f}"
    )


[2026-02-15 08:26:55] INFO modeling.py:987: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Response: {"question": "What is the next number in the series: 1, 4, 9, 16, 25, ___?", "A": "30", "B": "35", "C": "36", "D": "40"}
Total tokens: 61, Time taken: 3.23 seconds, TGPS: 18.86
